In [ ]:
!pip install streamlit --quiet
!pip install plotly.express
!npm install localtunnel --quiet
!pip install duckdb



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 516.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.6 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇
added 22 packages in 3s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇Requirement already satisfied: duckdb in /usr/local/lib/python3.11/dist-packages (1.1.3)


In [ ]:
%%writefile dashboard.py

import random
import pandas as pd
import streamlit as st
import plotly.express as px
import duckdb
import plotly.graph_objects as go

st.set_page_config(page_title="Desenrola Brasil Dashboard", page_icon=":bar_chart:", layout="wide")

st.title("Streamlit -- Dashboard - Desenrola Brasil")
st.markdown("Prototipo Dashboard")
st.markdown("---")

# Menu (lateral) - Upload de Arquivo
with st.sidebar:
    st.header("Configurações")
    uploaded_file = st.file_uploader("Escolha o arquivo Excel")

if uploaded_file is None:
    st.info("Carregue um arquivo via configurações", icon="ℹ️")
    st.stop()

# Carregando os dados
@st.cache_data
def load_data(path: str):
    df = pd.read_excel(path)
    return df

df = load_data(uploaded_file)

with st.expander("Pré-Visualização"):
    st.dataframe(df)

# Gráfico de Pizza
def plot_pizza():
    state_data = duckdb.sql(
        f"""
        SELECT
            UNIDADE_FEDERACAO,
            SUM(VOLUME_OPERACOES) AS TOTAL_VOLUME
        FROM df
        GROUP BY UNIDADE_FEDERACAO
        """
    ).df()

    fig = px.pie(
        state_data,
        names="UNIDADE_FEDERACAO",
        values="TOTAL_VOLUME",
        title="Distribuição do Volume de Operações por Unidade de Federação",
        color="UNIDADE_FEDERACAO",
        hole=0.3,
    )

    st.plotly_chart(fig, use_container_width=True)

# Gráfico de Barras Empilhadas
def plot_bar_stacked():
    stacked_data = duckdb.sql(
        f"""
        SELECT
            NOME_CONGLOMERADO_FINANCEIRO,
            UNIDADE_FEDERACAO,
            SUM(NUMERO_OPERACOES) AS TOTAL_OPERACOES,
            SUM(VOLUME_OPERACOES) AS TOTAL_VOLUME
        FROM df
        GROUP BY NOME_CONGLOMERADO_FINANCEIRO, UNIDADE_FEDERACAO
        """
    ).df()

    fig = px.bar(
        stacked_data,
        x="NOME_CONGLOMERADO_FINANCEIRO",
        y=["TOTAL_OPERACOES", "TOTAL_VOLUME"],
        color="UNIDADE_FEDERACAO",
        title="Operações e Volume por Conglomerado e Estado (Empilhado)",
        labels={"TOTAL_OPERACOES": "Número de Operações", "TOTAL_VOLUME": "Volume de Operações"},
        height=600,
        barmode="stack",
    )

    st.plotly_chart(fig, use_container_width=True)

# Gráfico de Dispersão (Scatter Plot)
def plot_scatter():
    scatter_data = df[["NUMERO_OPERACOES", "VOLUME_OPERACOES"]].dropna()

    fig = px.scatter(
        scatter_data,
        x="NUMERO_OPERACOES",
        y="VOLUME_OPERACOES",
        title="Relação entre Número de Operações e Volume de Operações",
        labels={"NUMERO_OPERACOES": "Número de Operações", "VOLUME_OPERACOES": "Volume de Operações"},
    )

    st.plotly_chart(fig, use_container_width=True)

# Gráfico de Barras Horizontal
def plot_bar_horizontal():
    horizontal_data = duckdb.sql(
        f"""
        SELECT
            UNIDADE_FEDERACAO,
            SUM(VOLUME_OPERACOES) AS TOTAL_VOLUME
        FROM df
        GROUP BY UNIDADE_FEDERACAO
        """
    ).df()

    fig = px.bar(
        horizontal_data,
        x="TOTAL_VOLUME",
        y="UNIDADE_FEDERACAO",
        orientation="h",
        title="Volume de Operações por Unidade de Federação",
        labels={"TOTAL_VOLUME": "Volume de Operações", "UNIDADE_FEDERACAO": "Unidade de Federação"},
        height=600,
    )

    st.plotly_chart(fig, use_container_width=True)

# Layout - Streamlit

# Exibir apenas o gráfico de pizza no topo central
top_center_column = st.columns(1)[0]

with top_center_column:
    plot_pizza()

# Exibir os gráficos restantes abaixo
bottom_left_column, bottom_right_column = st.columns(2)

with bottom_left_column:
    plot_bar_stacked()

with bottom_right_column:
    plot_scatter()

with st.expander("Gráfico de Barras Horizontal"):
    plot_bar_horizontal()


Writing dashboard.py


In [ ]:
!curl ipv4.icanhazip.com

35.245.14.47


In [ ]:
!streamlit run dashboard.py & npx localtunnel --port 8501



⠙
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.245.14.47:8501

your url is: https://silly-shirts-pay.loca.lt
2025-03-14 00:29:27.442 Uncaught app execution
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/runtime/scriptrunner/exec_code.py", line 121, in exec_func_with_error_handling
    result = func()
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 593, in code_to_exec
    exec(code, module.__dict__)
  File "/content/dashboard.py", line 129, in <module>
    plot_pizza()
  File "/content/dashboard.py", line 37, in plot_pizza
    state_data = duckdb.sql(
                 ^^^^^^^^^^^
duckdb.duckdb.BinderException: Binder Error: Referenced column "UNIDADE_FEDERACAO" not found in FROM clause!
Candidate bindings: "df.DATA", "df.IPCA", "df.TAXA_SELIC"
2025-03-14 0